In [1]:
!conda install -c conda-forge wikipedia --yes
#!pip install wikipedia
import pandas as pd
import wikipedia as wp

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wikipedia-1.4.0            |           py35_0          20 KB  conda-forge

The following NEW packages will be INSTALLED:

    wikipedia: 1.4.0-py35_0 conda-forge


wikipedia-1.4.0      | 20 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Data from wikipage

In [2]:
# Get the table from wiki page
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]

# Drope the rows with Borough of "Not assigned"
df_assigned = df[df.Borough != 'Not assigned']
df_assigned.reset_index(drop = True, inplace = True)

# Merge rows with the same Postcode
df_assigned = df_assigned.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

# Set 'Not assigned' Neigbourhood to be equal to Borough
na_nb_idx =df_assigned['Neighbourhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighbourhood'] = df_assigned[na_nb_idx].Borough
df_assigned[df_assigned['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [3]:
df_assigned.shape

(103, 3)

Add latitude and longitude of each neighborhood

In [4]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-05-15 15:37:58--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-05-15 15:37:58--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-05-15 15:38:01--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-05-15 15:38:02--  https://ibm.box.com/public/static/9afzr83pps4pwf2sm

In [5]:
df_cord = pd.read_csv('GeoCord.csv') # Read the csv data
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
import numpy as np

In [9]:

# Create Latitude and Longitude columns in df_assigned
df_assigned['Latitude'] = np.nan
df_assigned['Longitude'] = np.nan

# For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df_assigned.index:
    cord_idx = df_cord['Postal Code'] == df_assigned.loc[idx, 'Postcode']
    df_assigned.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df_assigned.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values

# Display the results
df_assigned

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


In [10]:
from geopy.geocoders import Nominatim

In [12]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


altair-2.2.2         | 462 KB    | #############################

Msrking Nighboorhood in MAP


In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_assigned['Latitude'], df_assigned['Longitude'], df_assigned['Borough'], df_assigned['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto